# Recommender system - a collaborative filtering approach

In this project we will work on a recommender system constructed through the **collaborative filtering** perpective. Collaborative filtering techniques find similar groups of users and provide recommendations based on similar tastes within that group as opposed to a **content-based approach**, in which we try to find similarities between items a user have already tried and then recommend new ones based on those features they share. Our measure of similarity is chosen to be the **Pearson correlation function** and our dataset consists of a list of movies and a corresponding list of ratings given by users retrieved from the site [MovieLens](https://grouplens.org/datasets/movielens). This project is based on [this article](https://medium.com/swlh/how-to-build-simple-recommender-systems-in-python-647e5bcd78bd). 

## Loading and preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
# Let us remove the year from the title and store it in a new column
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
# Removing parenthesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [5]:
# We won't need the genres columns, thus we are going to drop it
movies_df.drop('genres', axis=1, inplace = True)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [6]:
# Let us now take a look into the ratings dataset
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# We won't need the information contained in the timestamp column, therefore we are going to drop it
ratings_df.drop('timestamp', 1, inplace=True)
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


## Collaborative filtering 

Our aim in a first moment is to construct the recommender system using the collaborative filtering techniques in a **memory-based** approach, in which statistical tools (here, the Pearson correlation function) are used to approximate users or items based on historical data. Afterwards we are going to dive into a **model-based** analysis, where predictions for users preferences are made through the development of a machine learning model. 
Thus, our goal is to find users that have similar preferences to the input user and then recommend items that they have liked to the input. Let's begin by creating an input user to recommend movies to.

In [8]:
user_input = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
input_movies = pd.DataFrame(user_input)
input_movies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [9]:
# Adding the movie ID to the input_movies dataframe

inputId = movies_df[movies_df['title'].isin(input_movies['title'].tolist())]
input_movies = pd.merge(inputId, input_movies)
input_movies

,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


In [10]:
#Filtering out users that have watched the same movies as the input

user_subset = ratings_df[ratings_df['movieId'].isin(input_movies['movieId'].tolist())]
user_subset_grouped = user_subset.groupby('userId')
user_subset_grouped.head(20)

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0
...,...,...,...
99510,609,296,4.0
99534,610,1,5.0
99552,610,296,5.0
99636,610,1274,5.0


In [11]:
#Sorting the dataframe so users with most movies in common with the input will have priority

user_subset_grouped = sorted(user_subset_grouped, key=lambda x: len(x[1]), reverse=True)
#user_subset_grouped

### Pearson correlation function and similarity between instances and the input user

Next, we are going to compare all (almost) users to our specified user and find the one that is most similar.
we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. It is written as:

$$\large r=\frac{\sum_{i=1}^{n}(x_i-\bar x)(y_i-\bar y)}{\sqrt{\sum_{i=1}^{n}(x_i-\bar x)^2}\sqrt{\sum_{i=1}^{n}(y_i-\bar y)^2}}$$

It has the interesting property of being invariant under a change of scale. It goes from -1 to 1 such that, in our case, a 1 means that the two users have similar tastes while a -1 means the opposite. 

We will select a subset of users to iterate through as we don't want to go through all the users:

In [12]:
user_subset_grouped = user_subset_grouped[0:200]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the correlation coefficient.

In [13]:
from math import sqrt

pearson_correlation_dict={}
for name, group in user_subset_grouped:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    input_movies = input_movies.sort_values(by='movieId')
    #Get the N for the formula
    n_ratings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = input_movies[input_movies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format 
    temp_rating_list = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    temp_group_list = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list),2)/float(n_ratings)
    Syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list),2)/float(n_ratings)
    Sxy = sum( i*j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list)*sum(temp_group_list)/float(n_ratings)
    
    #If the denominator is different from zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearson_correlation_dict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearson_correlation_dict[name] = 0

In [14]:
pearsonDF = pd.DataFrame.from_dict(pearson_correlation_dict, orient='index')
pearsonDF.columns = ['similarityindex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head(10)

,similarityindex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298
5,0.937614,414
6,0.117202,474
7,0.438529,477
8,0.784465,480
9,0.080064,483


Now let's get the top 50 users that are most similar to the input.

In [16]:
top_users=pearsonDF.sort_values(by='similarityindex', ascending=False)[0:50]
top_users.head()

,similarityindex,userId
43,1.0,132
100,1.0,5
144,1.0,191
129,1.0,125
130,1.0,130


Now we are at the point where we can recommend movies to the input user. First we need to get the movies watched by the top users in our pearsonDF from the ratings dataframe. 

In [17]:
top_users_rating=top_users.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
top_users_rating.head()

,similarityindex,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


We are going to perform a weighted average of the ratings of the movies using the Pearson Correlation as the weight.

In [18]:
#Multiplies the similarity by the user's ratings
top_users_rating['weighted_rating'] = top_users_rating['similarityindex']*top_users_rating['rating']
#Applies a sum to the top_users after grouping it up by userId
temp_top_users_rating = top_users_rating.groupby('movieId').sum()[['similarityindex','weighted_rating']]
temp_top_users_rating.columns = ['sum_similarity_index','sum_weighted_rating']
temp_top_users_rating.head()

,sum_similarity_index,sum_weighted_rating
movieId,,
1,38.599046,135.142614
2,20.674256,59.247476
3,7.728581,26.157351
5,4.898383,13.796767
6,13.827596,53.863790


In [19]:
#Creating an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = temp_top_users_rating['sum_weighted_rating']/temp_top_users_rating['sum_similarity_index']
recommendation_df['movieId'] = temp_top_users_rating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.501191,1
2,2.865761,2
3,3.384496,3
5,2.816596,5
6,3.895384,6


Now let's sort it and see the top 10 movies that the algorithm recommended:

In [23]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())].reset_index(drop=True)

,movieId,title,year
0,85,Angels and Insects,1995
1,4117,Hope and Glory,1987
2,6345,"Chorus Line, A",1985
3,6461,"Unforgiven, The",1960
4,7614,Oklahoma!,1955
5,32892,Ivan's Childhood (a.k.a. My Name is Ivan) (Iva...,1962
6,112818,"Purge: Anarchy, The",2014
7,140265,George Carlin: Jammin' in New York,1992
8,172909,Cheburashka,1971
9,173351,Wow! A Talking Fish!,1983


## Model-based analysis through unsupervised learning